In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import pearsonr
import os
from sklearn.cluster import KMeans
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
# from utils import Draw_fig 
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

In [3]:
def extrtact_features(mse, Pxx_den):
    # 均值 方差
    f_m=np.mean(Pxx_den)
    f_std= np.std(Pxx_den)
    # 四分位距number of outlier
    Q1 = np.percentile(Pxx_den, 25)
    Q3 = np.percentile(Pxx_den, 75)
    IQR = Q3 - Q1
    num_outlier = sum((Pxx_den < (Q1 - 1.5 * IQR)) | (Pxx_den > (Q3 + 1.5 * IQR)))
    #MSE
    f_mse=sum(np.square(Pxx_den-np.mean(Pxx_den)))/len(Pxx_den)
    
    p_mse = sum(np.square(Pxx_den - np.mean(Pxx_den))) / len(Pxx_den)
    
    #Max diff
    f_md=np.max(Pxx_den)-np.min(Pxx_den)
    return mse,f_m,f_std,IQR,num_outlier,p_mse,f_mse,f_md

def save_fig(ex, ey, ez, path, lim=0.01, yline=0.0025):
    x_lim=np.arange(0,len(ex),1)
    fig, [ax1, ax2, ax3] = plt.subplots(3, 1, sharex=True,figsize=(8, 7))

    # ax1.xaxis.set_major_locator(xmajorLocator)
    ax1.axhline(y=yline, c="green",  linewidth=0.8)
    ax1.axhline(y=-yline, c="green",  linewidth=0.8)
    ax1.plot(x_lim, ex, linewidth=0.5)
    ax1.set_ylim((-0.01, 0.01))
    ax1.set_ylabel('X', fontsize=18)
    ax1.grid(True)

    # ax2.xaxis.set_major_locator(xmajorLocator)
    ax2.axhline(y=yline, c="green",  linewidth=0.8)
    ax2.axhline(y=-yline, c="green",  linewidth=0.8)
    ax2.plot(x_lim, ey, linewidth=0.5)
    ax2.set_ylim((-0.01, 0.01))
    ax2.set_ylabel('Y', fontsize=18)
    ax2.grid(True)

    # ax3.xaxis.set_major_locator(xmajorLocator)
    ax3.axhline(y=yline, c="green",  linewidth=0.8)
    ax3.axhline(y=-yline, c="green",  linewidth=0.8)
    ax3.plot(x_lim, ez, linewidth=0.5)
    ax3.set_ylim((-0.01, 0.01))
    ax3.set_ylabel('Z', fontsize=18)
    ax3.grid(True)
    plt.tight_layout()
    plt.savefig(path, dpi=100)
    plt.clf()
    plt.close()

In [1]:
fs=100
path='/home/Amin/EQ_Place/dataset/EQ_20210829_130323'
files=os.listdir(path)
files.sort()

In [4]:
len(files)

2714

In [6]:
features=[]
data=[]
for i in tqdm(range(len(files))):
    f=files[i]
    #read data and substract DC offset
    df=pd.read_csv(path+f)
    df=np.array(df)[:,:3]
    ex,ey,ez=df[:,0],df[:,1],df[:,2]

    ex_mse = sum(np.square(ex - np.mean(ex))) / len(ex)
    ey_mse = sum(np.square(ey - np.mean(ey))) / len(ey)
    ez_mse = sum(np.square(ez - np.mean(ez))) / len(ez)
    
    ex,ey,ez=ex-np.mean(ex), ey-np.mean(ey), ez-np.mean(ez)
    sos = signal.butter(2, 35, 'lowpass',fs=100,output='sos')
    ex = signal.sosfilt(sos, ex)
    ey = signal.sosfilt(sos, ey)
    ez = signal.sosfilt(sos, ez)

    data.append([ex,ey,ez])
    
    #calculate PSD
    fre, Pxx_den = signal.periodogram(ex, fs)
    Pxx_den=10*np.log10(abs(Pxx_den))[1:]
    
    fre, Pyy_den = signal.periodogram(ey, fs)
    Pyy_den=10*np.log10(abs(Pyy_den))[1:]
    
    fre, Pzz_den = signal.periodogram(ez, fs)
    Pzz_den=10*np.log10(abs(Pzz_den))[1:]
    
    fx=extrtact_features(ex_mse, Pxx_den)
    fy=extrtact_features(ey_mse, Pyy_den)
    fz=extrtact_features(ez_mse, Pzz_den)
    
    features.append([fx,fy,fz])
    
data=np.array(data)
features=np.array(features)
features=features.reshape(features.shape[0],-1)

100%|██████████| 2714/2714 [26:45<00:00,  1.69it/s]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
#聚类： 先异常值处理 进行归一化再进行聚类
idx = np.isinf(features)
features[idx] = 0 
idx = np.isnan(features)
features[idx] = 0 

ss = StandardScaler()
z_f = ss.fit_transform(features)

kmeans_model = KMeans(n_clusters=2, random_state=1).fit(z_f)
labels = kmeans_model.labels_

In [ ]:
pd.value_counts(labels)

In [9]:
np.save('labels_8_29',labels)

In [30]:
if not os.path.exists('../processing/3_15'):
    os.mkdir('../processing/3_15')

In [31]:
if not os.path.exists('../processing/3_15/n'):
    os.mkdir("../processing/3_15/n")
if not os.path.exists('../processing/3_15/p'):
    os.mkdir("../processing/3_15/p")
for i in tqdm(range(len(files))):
    ex,ey,ez=data[i, 0, :], data[i, 1, :], data[i, 2, :]
    
    if labels[i] ==1:
        filename='C:/ysz/processing/3_15/p/'+files[i][:-4]
        save_fig(ex,ey,ez,filename)
    else:
        filename='C:/ysz/processing/3_15/n/'+files[i][:-4]
        save_fig(ex,ey,ez,filename)

100%|██████████| 508/508 [05:08<00:00,  1.64it/s]
